In [1]:
import os
os.getcwd()

'/Volumes/GoogleDrive/Shared drives/Lauren and Hilary/Regional Overland Flow Project/Data and Scripts/Regional-OF'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
os.chdir('../pysheds_trial/pysheds-master/')

In [5]:
from pysheds.grid import Grid

In [9]:
grid = Grid.from_raster('../hyd_na_dem_30s/hyd_na_dem_30s.tif')

TypeError: `nodata` value not representable in dtype of array.